#### Проект 4. Решение комплексной бизнес-задачи

**ЦЕЛЬ**

Подготовить основу рекомендательной системы.

**КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)**

1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
2. Обработать данные средствами Python.
3. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.

**Раздел 1.** Подготовка данных с помощью SQL

Для выгрузки данных по продажам курсов в разрезе пользователей (купивших более одного курса) был составлен следующий SQL-запрос:


WITH more_then_one_course AS  
(SELECT  
    user_id,  
    COUNT (DISTINCT resource_id) as courses  
FROM     
    final.carts  
    JOIN final.cart_items AS items ON carts.id=items.cart_id  
WHERE resource_type='Course' and state='successful'  
GROUP BY 1    
HAVING COUNT (DISTINCT resource_id)>1)  

SELECT   
    DISTINCT carts.user_id,  
    resource_id  
FROM   
    final.carts  
    JOIN final.cart_items AS items ON carts.id=items.cart_id  
    JOIN more_then_one_course AS course ON course.user_id=carts.user_id   
WHERE resource_type='Course' and state='successful'  
ORDER BY 1  

В результате была получена таблица из двух столбцов: ID пользователя и ID приобритенного курса.

**Раздел 2.** Обработка данных в Python

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations

In [2]:
df = pd.read_csv('data/final_project_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [3]:
# Создаем датафрейм со списком курсов для каждого пользователя.
purchase_list = df.groupby('user_id')['resource_id'].apply(lambda x:list(np.unique(x))).reset_index()

In [4]:
# Получаем датафрейм с 12656 клиентами.
purchase_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12656 entries, 0 to 12655
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      12656 non-null  int64 
 1   resource_id  12656 non-null  object
dtypes: int64(1), object(1)
memory usage: 197.9+ KB


In [5]:
# Разбиваем все покупки курсов по парам для каждого клиента.
course_combinations = purchase_list['resource_id'].apply(lambda x: list(combinations(x,2)))
course_combinations.to_frame()

,resource_id
0,"[(516, 1099)]"
1,"[(356, 357), (356, 1125), (357, 1125)]"
2,"[(553, 1147)]"
3,"[(361, 1138)]"
4,"[(356, 357)]"
...,...
12651,"[(566, 750)]"
12652,"[(794, 864), (794, 1129), (864, 1129)]"
12653,"[(356, 553), (356, 571), (356, 765), (356, 912..."
12654,"[(515, 743)]"


In [6]:
# Создаем множество для нахождения количества различных пар курсов в покупках клиентов.
courses_list = course_combinations.tolist()
combinations_set = set()
for item in courses_list:
    for i in item:
        combinations_set.add(i)
print("Количество различных пар курсов:", len(combinations_set))

Количество различных пар курсов: 3989


In [7]:
# Создаем список из пар курсов, затем используем счетчик Counter для определения количества каждой пары в покупках.
pairs_list = list()
for item in courses_list:
    for i in item:
        pairs_list.append(i)

In [8]:
from collections import Counter
c = Counter(pairs_list)
print(c)

Counter({(551, 566): 797, (515, 551): 417, (489, 551): 311, (523, 551): 304, (566, 794): 290, (489, 515): 286, (490, 566): 253, (490, 551): 247, (570, 752): 247, (569, 572): 216, (515, 523): 213, (553, 745): 212, (489, 523): 206, (569, 840): 204, (514, 551): 200, (516, 745): 199, (515, 566): 195, (489, 566): 188, (504, 572): 184, (572, 840): 178, (551, 552): 177, (507, 570): 172, (490, 809): 163, (489, 490): 152, (507, 752): 150, (523, 552): 144, (490, 515): 143, (551, 570): 142, (504, 569): 139, (514, 515): 139, (551, 745): 138, (514, 566): 138, (502, 551): 135, (504, 840): 135, (571, 1125): 122, (502, 566): 120, (523, 566): 120, (570, 809): 119, (752, 809): 115, (490, 523): 114, (357, 571): 112, (523, 564): 110, (551, 749): 109, (516, 553): 107, (551, 777): 107, (551, 679): 104, (356, 571): 103, (551, 564): 103, (515, 749): 103, (568, 745): 102, (356, 357): 100, (363, 511): 99, (551, 571): 98, (551, 809): 96, (502, 514): 95, (551, 794): 95, (490, 514): 94, (566, 764): 92, (490, 564):

В результате обработки данных было получено, что в покупках клиентов, которые приобрели более одного курса, встречается 3989 различных пар курсов. Наиболее популярной является пара, состоящая из курсов 551 и 566.

**Раздел 3.** Создание рекомендательной таблицы

In [9]:
# Создаем словарь, в котором ключами являются уникальные пары курсов, а значения - количество покупок этих пар.
courses_dict = dict(c)
courses_dict

{(516, 1099): 25,
 (356, 357): 100,
 (356, 1125): 44,
 (357, 1125): 52,
 (553, 1147): 16,
 (361, 1138): 40,
 (1125, 1140): 1,
 (551, 745): 138,
 (553, 745): 212,
 (551, 1138): 14,
 (553, 568): 83,
 (514, 517): 10,
 (514, 566): 138,
 (517, 566): 21,
 (363, 511): 99,
 (363, 562): 77,
 (363, 563): 33,
 (511, 562): 55,
 (511, 563): 19,
 (562, 563): 53,
 (568, 745): 102,
 (509, 553): 48,
 (509, 745): 59,
 (1125, 1144): 22,
 (509, 568): 46,
 (509, 672): 5,
 (568, 672): 4,
 (516, 552): 12,
 (356, 552): 7,
 (357, 571): 112,
 (509, 516): 35,
 (516, 568): 54,
 (513, 1141): 34,
 (571, 1125): 122,
 (551, 552): 177,
 (551, 744): 16,
 (551, 862): 8,
 (552, 744): 8,
 (552, 862): 6,
 (552, 1138): 4,
 (744, 862): 2,
 (744, 1138): 1,
 (862, 1138): 3,
 (356, 679): 8,
 (571, 745): 22,
 (571, 1099): 2,
 (745, 1099): 53,
 (509, 1099): 31,
 (568, 1099): 53,
 (517, 750): 34,
 (800, 1125): 4,
 (569, 840): 204,
 (745, 1125): 15,
 (509, 514): 8,
 (509, 551): 15,
 (514, 551): 200,
 (514, 745): 38,
 (571, 765): 83

In [10]:
# Создаем датафрейм с номерами всех курсов и частотой их покупок. Таким образом определяем самые популярные курсы.
course_recommend_df = df['resource_id'].value_counts().to_frame().reset_index()
course_recommend_df.columns = ['course_ID','qty_of_purchase']
course_recommend_df.head()

,course_ID,qty_of_purchase
0,551,2935
1,566,2342
2,515,1311
3,489,1125
4,490,1084


In [11]:
# Определяем границу значений по покупкам курсов для 90% процентиля. 
# Таким образом, для сочетаний курсов, которые были куплены в меньшем количестве, чем определяемая величина
# min_value, будут предложены альтернативные курсы 551 (как самый популярный) и 566 (второй по популярности).

courses_df = pd.DataFrame.from_dict(courses_dict,orient='index').rename(columns={0:'qty'}).reset_index()

min_value = np.percentile(courses_df['qty'],90)
print('Минимальная граница для частотности рекомендаций:', min_value)

Минимальная граница для частотности рекомендаций: 22.0


In [12]:
# Составляем функцию, которая будет подбирать для каждого номера курса пару рекомендаций.
def recommendation(id):
    recommendation_list = []
    for i in courses_dict.keys():
        if i[0] == id:
            recommendation_list.append((i,courses_dict[i]))
        elif i[1] == id:
            recommendation_list.append((i,courses_dict[i]))
    recommendation_list_sorted = sorted(recommendation_list, key=lambda x: x[1], reverse=True)
    return recommendation_list_sorted[:2]

In [13]:
# Создаем новый датафрейм со столбцами для рекомендаций и составляем цикл для заполнения этого датафрейма. 
# Если рекомендуемый курс был приобретен большее количество раз, чем значение минимальной границы частотности, 
# то в рекомендациях записывается этот курс. Если же курс был приобретен меньшее количество раз, то будут 
# предложены курсы 551 и 566.

recommendation_list = []
recommend_df = pd.DataFrame(recommendation_list, columns=['recom_1','recom_2'])

course_list = df['resource_id'].unique()

for course_id in course_list:
    rec1 = None
    rec2 = None
    if recommendation(course_id)[0][1] >= min_value:
        rec1 = (set(recommendation(course_id)[0][0])-set([course_id])).pop()
    if recommendation(course_id)[1][1] >= min_value:
        rec2 = (set(recommendation(course_id)[1][0])-set([course_id])).pop()
    if (recommendation(course_id)[0][1] >= min_value) and (recommendation(course_id)[1][1] < min_value):
        rec2 = 551
    if recommendation(course_id)[0][1] < min_value:
        rec1 = 551
        rec2 = 566
    recommend_df.loc[course_id] = [rec1, rec2]
    
recommend_df = recommend_df.sort_index().reset_index()
recommend_df.columns = ['course_id','recom_1','recom_2']
recommend_df

,course_id,recom_1,recom_2
0,356,571,357
1,357,571,356
2,358,570,551
3,359,551,566
4,360,551,566
...,...,...,...
121,1188,551,566
122,1198,551,566
123,1199,551,566
124,1200,551,566


In [14]:
# Выгружаем данные из датафрейма recommend_df в csv-файл и сохраняем файл в папке data
recommend_df.to_csv(
    'data/recommendation_table.csv', index=False, sep=' '
)